In [1]:
import numpy as np
import random
import copy
import math
from tqdm import tqdm
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from dataset import TimeSeriesDataset, TimeSeriesDatasetConcat
from martins.complex_transformer import ComplexTransformer
import argparse

In [4]:
class Generator(nn.Module):
    def __init__(self, feature_dim):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            #nn.Tanh()
        ) 

    def forward(self, x):
        # x: [bs, seq, init_size (small)]
        return self.net(x)


In [5]:
class Discriminator(nn.Module):
    def __init__(self, feature_dim, d_out):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(

            nn.Linear(feature_dim, feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),
        ) 
        self.fc = nn.Linear(3200, 1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        # x: [bs, seq, feature_dim]
        x = self.net(x)
        bs = x.shape[0]
        x = x.reshape(bs, -1)
        out = self.sigmoid(self.fc(x))
        return out


In [6]:
class FNN(nn.Module):
    def __init__(self, d_in, d_h, d_out, dp):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(d_in, d_h)
        self.fc2 = nn.Linear(d_h, d_out)
        self.dp = nn.Dropout(dp)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp(x)
        x = self.fc2(x)

        return x


In [3]:
def get_source_dict(file_path, num_class, data_len=None):
    '''
    output:
        {class: [data]},
        data_len
    '''
    data_ = np.load(file_path, allow_pickle=True)
    train_data = data_['tr_data']
    train_lbl = data_['tr_lbl']
    if data_len:
        train_data = data_['tr_data'][:data_len]
        train_lbl = data_['tr_lbl'][:data_len]
    data_dict = get_class_data_dict(train_data, train_lbl, num_class)
    
    return data_dict, train_data.shape[0]

def get_target_dict(file_path, num_class, lbl_percentage, seed=0):
    '''
    split target domain data
    
    output:
        with label:
            {class: [data]}
        without label:
            [data], [lbl]
        data_len
    '''
    data_ = np.load(file_path, allow_pickle=True)
    train_data = data_['te_data']
    train_lbl = data_['te_lbl']
    
    np.random.seed(seed)
    index = np.random.permutation(train_data.shape[0])
    train_data = train_data[index]
    train_lbl = np.argmax(train_lbl[index], -1)

    with_label = {i:[] for i in range(num_class)}
    labeled_index = []
    for i in with_label:
        index = np.argwhere(train_lbl==i).flatten()
        np.random.seed(seed)
        index = np.random.choice(index, int(lbl_percentage*train_lbl.shape[0]/num_class))
        labeled_index.extend(index)
        with_label[i] = train_data[index]

    
    return with_label, (np.delete(train_data,labeled_index,axis=0), np.delete(train_lbl,labeled_index,axis=0)), (train_data[labeled_index], train_lbl[labeled_index]), train_data.shape[0]

def get_class_data_dict(data, lbl, num_class):
    '''
    construct a dict {label: data}  
    '''
    lbl_not_one_hot = np.argmax(lbl, -1)
    result = {i:[] for i in range(num_class)}
    for i in result:
        index = np.argwhere(lbl_not_one_hot==i).flatten()
        result[i] = data[index]
        
    return result

def get_batch_source_data_on_class(class_dict, num_per_class):
    '''
    get batch from source data given a required number of sample per class
    '''
    batch_x = []
    batch_y = []
    for key, value in class_dict.items():
        index = random.sample(range(len(value)), num_per_class)
        batch_x.extend(value[index])
        batch_y.extend([key] * num_per_class)
        
    return np.array(batch_x), np.array(batch_y)

def get_batch_target_data_on_class(real_dict, pesudo_dict, unlabel_data, num_per_class, compromise=1, real_weight=1, pesudo_weight=0.5):
    '''
    get batch from target data given a required number of sample per class
    '''
    batch_x = []
    batch_y = []
    batch_real_or_pesudo = []
    for key in real_dict:
        real_num = len(real_dict[key])
        pesudo_num = len(pesudo_dict[key])
        num_in_class = real_num + pesudo_num
        
        if num_in_class < num_per_class:
            # if totoal number sample in this class is less than the required number of sample
            # then fetch the remainding data randomly from the unlabeled set with a compromise
            
            num_fetch_unlabeled = (num_per_class - num_in_class) * compromise
            index = random.sample(range(unlabel_data.shape[0]), num_fetch_unlabeled)
            batch_x.extend(unlabel_data[index])
            batch_y.extend([key] * num_fetch_unlabeled)
            batch_real_or_pesudo.extend([pesudo_weight] * num_fetch_unlabeled)
            
            batch_x.extend(real_dict[key])
            batch_real_or_pesudo.extend([real_weight] * real_num)
            batch_x.extend(pesudo_dict[key])
            batch_real_or_pesudo.extend([pesudo_weight] * pesudo_num)
            batch_y.extend([key] * num_in_class)
            
        else:
            index = random.sample(range(num_in_class), num_per_class)
            index_in_real = []
            index_in_pesudo = []
            for i in index:
                if i >= real_num:
                    index_in_pesudo.append(i-real_num)
                else:
                    index_in_real.append(i)
                    
            batch_x.extend(real_dict[key][index_in_real])
            batch_real_or_pesudo.extend([real_weight] * len(index_in_real))
            batch_x.extend(pesudo_dict[key][index_in_pesudo,:])
            batch_real_or_pesudo.extend([pesudo_weight] * len(index_in_pesudo))
            batch_y.extend([key] * num_per_class)
    
    return np.array(batch_x), np.array(batch_y), np.array(batch_real_or_pesudo)



In [48]:
#local only

class fake_args():
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

        
# args = fake_args(data_path='../data_unzip/', 
#                  task='3A', 
#                  batch_size=100,
#                  epochs=10,
#                  lr_gan=1e-3,
#                  lr_clf=1e-3,
#                  gap=2,
#                  lbl_percentage=0.2,
#                  num_per_class=-1,
#                  seed=0,
#                  save_path='../train_related/JDA_GAN',
#                  model_save_period=1,
#                  classifier='/Users/stevenliu/time-series-adaption/time-series-domain-adaptation/JDA/FNN_trained_model'
#                  )


args = fake_args(data_path='/home/weixinli/',
                  task='3E',
                  batch_size=100,
                  epochs=800,
                  gap=4,
                  lbl_percentage=1,
                  num_per_class=10,
                  seed=0,
                  classifier='/home/weixinli/time-series-domain-adaptation/JDA/FNN_trained_model')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if args.seed is None:
    args.seed = random.randint(1, 10000)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
np.random.seed(args.seed)
cudnn.deterministic = True
torch.backends.cudnn.deterministic = True

args.task = '3Av2' if args.task == '3A' else '3E'
d_out = 50 if args.task == "3Av2" else 65
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if args.num_per_class == -1:
    args.num_per_class = math.ceil(args.batch_size / d_out)
    
model_sub_folder = '/task_%s_gap_%s_lblPer_%1.1f_numPerClass_%i'%(args.task, args.gap, args.lbl_percentage, args.num_per_class)

In [49]:
target_dict, (target_unlabel_x, target_unlabel_y), (target_label_x, target_label_y), target_len  = get_target_dict(args.data_path+'processed_file_%s.pkl'%args.task, d_out, args.lbl_percentage)
source_dict, source_len = get_source_dict(args.data_path+'/processed_file_%s.pkl'%args.task, d_out, data_len=target_len)

seq_len = 10 
feature_dim = 160
classifier_model_folder = 'Final_FNN_' + args.task 
CNet_path = args.classifier + '/' + classifier_model_folder + "/CNet_model.ep100"
encoder_path = args.classifier + '/' + classifier_model_folder + "/Encoder_model.ep100"

criterion = nn.CrossEntropyLoss()

CNet = FNN(d_in=feature_dim * 2 * seq_len, d_h=500, d_out=d_out, dp=0.5)

encoder = ComplexTransformer(layers=1,
                       time_step=seq_len,
                       input_dim=feature_dim,
                       hidden_size=512,
                       output_dim=512,
                       num_heads=8,
                       out_dropout=0.5)

if torch.cuda.is_available():
    CNet.load_state_dict(torch.load(CNet_path))
    encoder.load_state_dict(torch.load(encoder_path))
else:
    CNet.load_state_dict(torch.load(CNet_path, map_location=torch.device('cpu')))
    encoder.load_state_dict(torch.load(encoder_path, map_location=torch.device('cpu')))
encoder.to(device)
CNet.to(device)

def classifier_inference(encoder, CNet, x, y, x_mean_tr, x_std_tr, batch_size):
    CNet.eval()
    encoder.eval()
    with torch.no_grad():
        #normalize data
        x = (x - x_mean_tr) / x_std_tr
        # take the real and imaginary part out
        real = x[:,:,0].reshape(batch_size, seq_len, feature_dim).float()
        imag = x[:,:,1].reshape(batch_size, seq_len, feature_dim).float()
        if torch.cuda.is_available():
            real.to(device)
            imag.to(device)
        real, imag = encoder(real, imag)
        pred = CNet(torch.cat((real, imag), -1).reshape(x.shape[0], -1))
        loss = criterion(pred, y)

    return pred, loss


real_label = 0.99 # target domain
fake_label = 0.01 # source domain

feature_dim_joint = 2 * feature_dim
DNet_global = Discriminator(feature_dim=feature_dim_joint, d_out=d_out).to(device)
DNet_local = Discriminator(feature_dim=feature_dim_joint, d_out=d_out).to(device)
GNet = Generator(feature_dim=feature_dim_joint).to(device)

# TODO: add global & local loss
criterion_gan_global = nn.BCELoss()
criterion_gan_local = nn.BCELoss(reduction='none')

feature_dim_joint = 2 * feature_dim
joint_set = TimeSeriesDatasetConcat(root_dir=args.data_path, file_name='processed_file_%s.pkl'%args.task, seed=args.seed)
joint_loader = DataLoader(joint_set, batch_size=args.batch_size, shuffle=True)
total_error_D = total_error_G = 0

source_mean = joint_set.tr_data_mean
target_mean = joint_set.te_data_mean
source_std = joint_set.tr_data_std
target_std = joint_set.te_data_std

D_global_losses = []
D_local_losses = []
G_losses = []
unlabel_acces = []
unlabel_losses = []
label_acces = []
label_losses = []
combine_acces = []
combine_losses = []
print(device)

model_sub_folder = '/task_%s_gap_%s_lblPer_%1.1f_numPerClass_%i/'%(args.task, args.gap, args.lbl_percentage, args.num_per_class)
model_Gan_path = '/home/weixinli/time-series-domain-adaptation/train_related/JDA_GAN'
GNet_path = model_Gan_path+model_sub_folder+'GNet_795'
DNet_global_path=model_Gan_path+model_sub_folder+'DNet_global_795'
DNet_local_path= model_Gan_path+model_sub_folder+'DNet_local_795'

if torch.cuda.is_available():
    GNet.load_state_dict(torch.load(GNet_path))
    DNet_global.load_state_dict(torch.load(DNet_global_path))
    DNet_local.load_state_dict(torch.load(DNet_local_path))
else:
    GNet.load_state_dict(torch.load(GNet_path, map_location=torch.device('cpu')))
    DNet_global.load_state_dict(torch.load(DNet_global_path, map_location=torch.device('cpu')))
    DNet_local.load_state_dict(torch.load(DNet_local_path, map_location=torch.device('cpu')))
GNet.to(device)
DNet_local.to(device)
DNet_global.to(device)

unlabel_correct_target = 0.0
unlabel_loss = 0.0
target_pesudo_y = []

for batch in range(math.ceil(target_unlabel_x.shape[0]/args.batch_size)):
    batch_size = target_unlabel_x[batch*args.batch_size:(batch+1)*args.batch_size].shape[0]
    if batch_size == 0:
        continue
    target_unlabel_x_batch = torch.tensor(target_unlabel_x[batch*args.batch_size:(batch+1)*args.batch_size], device=device).float()
    target_unlabel_y_batch = torch.tensor(target_unlabel_y[batch*args.batch_size:(batch+1)*args.batch_size], device=device)
    target_unlabel_x_batch = target_unlabel_x_batch.reshape(batch_size, seq_len, feature_dim_joint)
    target_unlabel_x_batch = (target_unlabel_x_batch - target_mean) / target_std
    target_unlabel_x_batch_transform = GNet(target_unlabel_x_batch).reshape(batch_size, -1, 2)
    pred, loss = classifier_inference(encoder, CNet, target_unlabel_x_batch_transform, target_unlabel_y_batch, target_mean, target_std, batch_size)
    unlabel_correct_target += (pred.argmax(-1) == target_unlabel_y_batch).sum().item()
    unlabel_loss += loss.item()
    target_pesudo_y.extend(pred.argmax(-1).cpu().numpy())

label_correct_target = 0.0
label_loss = 0.0
for batch in range(math.ceil(target_label_x.shape[0]/args.batch_size)):
    batch_size = target_label_x[batch*args.batch_size:(batch+1)*args.batch_size].shape[0]
    if batch_size == 0:
        continue
    target_label_x_batch = torch.tensor(target_label_x[batch*args.batch_size:(batch+1)*args.batch_size], device=device).float()
    target_label_y_batch = torch.tensor(target_label_y[batch*args.batch_size:(batch+1)*args.batch_size], device=device)
    target_label_x_batch = target_label_x_batch.reshape(batch_size, seq_len, feature_dim_joint)
    target_label_x_batch = (target_label_x_batch - target_mean) / target_std
    target_label_x_batch_transform = GNet(target_label_x_batch).reshape(batch_size, -1, 2)
    pred, loss = classifier_inference(encoder, CNet, target_label_x_batch_transform, target_label_y_batch, target_mean, target_std, batch_size)
    label_correct_target += (pred.argmax(-1) == target_label_y_batch).sum().item()
    label_loss += loss.item()
    target_pesudo_y.extend(pred.argmax(-1).cpu().numpy()) 

target_pesudo_y = np.array(target_pesudo_y)
pesudo_dict = get_class_data_dict(target_unlabel_x, target_pesudo_y, d_out)
if args.lbl_percentage == 1:
    unlabel_acc_ = 0
    unlabel_loss_ = 0
else:
    unlabel_acc_ = unlabel_correct_target/target_unlabel_x.shape[0]
    unlabel_loss_ = unlabel_loss/target_unlabel_x.shape[0]

if args.lbl_percentage == 0: 
    label_acc_ = 0
    label_loss_ = 0
else:
    label_acc_ = label_correct_target/target_label_x.shape[0]
    label_loss_ = label_loss/target_label_x.shape[0]

combine_acc_ = (unlabel_correct_target+label_correct_target)/(target_unlabel_x.shape[0]+target_label_x.shape[0])
combine_loss_ = (label_loss+unlabel_loss)/(target_unlabel_x.shape[0]+target_label_x.shape[0])
print('Classifier: Unlabel acc: %f, loss: %f; Label acc: %f, loss: %f; Combine acc: %f, loss: %f'%(unlabel_acc_,unlabel_loss_,label_acc_,label_loss_,combine_acc_,combine_loss_))

unlabel_acces.append(unlabel_acc_)
unlabel_losses.append(unlabel_loss_)
label_acces.append(label_acc_)
label_losses.append(label_loss_)
combine_acces.append(combine_acc_)
combine_losses.append(combine_loss_)

cpu
Classifier: Unlabel acc: 0.000000, loss: 0.000000; Label acc: 0.016090, loss: 0.101672; Combine acc: 0.015692, loss: 0.101908
